In [ ]:
import os
import math

import cv2

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches

from uncertainties import ufloat
from uncertainties.umath import acos, sqrt

%matplotlib inline

In [ ]:
DATA_PATH = "../data/sunspots/"
TEST_FILE = "sunspots_1024_20130326.jpg"

In [ ]:
# global variables
# crop rectangular coordinates and width + height
x, y, h, w = 74, 71, 875, 875

# sun radius
r_sun = 696342 # km

# per pixel (rounded) resolution in km
px_w, px_h = round(2 * r_sun / w), round(2 * r_sun / h)
print("Horizontale und vertikale Auflösung in km: ", px_w, px_h)

In [ ]:
def read_file(path, file_name, show=False):
    img = cv2.imread(os.path.join(path, file_name), 0)
    
    fig, ax = plt.subplots(1, figsize=(10,10))

    rect = patches.Rectangle((x,y), w, h, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    
    plt.imshow(img)
    fig.savefig(f"check_scale_{file_name}")
    if not show:
        fig.clf()

    return img

In [ ]:
def get_circles(img, file_name, show=False):
    img = img[y:y+h, x:x+w]
    img = cv2.medianBlur(img,5)
    
    cimg = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,10,param1=50,param2=10,minRadius=0,maxRadius=20)

    annotations = []
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
            # draw the outer circle
            cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)
            annotations.append([i[0], i[1]])
    else:
        circles = []
    
    f, axs = plt.subplots(figsize=(20,20))
    plt.subplot(121)
    plt.imshow(img, cmap = 'gray')
    plt.title('Original Image')
    plt.xticks([]), plt.yticks([])
    plt.subplot(122)
    plt.imshow(cimg, cmap = 'gray')
    plt.title('Modified Image'), plt.xticks([]), plt.yticks([])
    plt.annotate(f"+ (0,0)", (w / 2, h / 2), color="red")
    for a in annotations:
        plt.annotate(f"({a[0]}, {a[1]})", (a[0], a[1]))

    plt.savefig(f"circles_{file_name}")
    if not show:
        plt.clf()

    return circles

In [ ]:
def calc_phi(circles, file_name):
    if len(circles) > 0:
        phis = []
        for idx, circle in enumerate(circles[0]):
            print(f"> Circle #{idx}")
            (x_px, y_px, r_px) = (circle[0], circle[1], circle[2])
            print(f"Pixel coordinates (x, y, r)=({x_px}px, {y_px}px, {r_px}px)")
            # set (0, 0) to center of sun
            (x, y, r) = (x_px - w / 2, y_px - h / 2, r_px)
            print(f"Centered pixel coordinates (x, y, r)=({x}px, {y}px, {r}px)")
            # transform from index position to position/distance in km
            (x, y, r) = (x * px_w, y * px_h, r * px_w)
            print(f"Centered scaled coordinates (x, y, r)=({x:.0f}km, {y:.0f}km, {r:.0f}km)")
            
            x_unc = ufloat(x, r)
            y_unc = ufloat(y, r)
            print(f"(x, y)-coordinates with radius as uncertainty: ({x_unc}, {y_unc})")
        
            # effective radius at sunspot latitude
            #r_eff_ss = math.sqrt(r_sun**2 - y**2)
            #print(f"Effective radius at sunspot latitude: {r_eff_ss} km")
            r_eff_ss_unc = sqrt(r_sun**2 - y_unc**2)
            print(f"Effective radius at sunspot latitude: {r_eff_ss_unc} km")
            
            # phi
            if abs(x_unc) > abs(r_eff_ss_unc):
                print(f"WARN: X greater than eff. radius (x={x_unc}, r_eff={r_eff_ss_unc})")
                #phi = np.nan
                phi_unc = ufloat(np.nan, np.nan)
            else:
                #phi = math.acos(x / r_eff_ss)
                phi_unc = acos(x_unc / r_eff_ss_unc)
        
            print(f"Phi={phi_unc}")
            phis.append([x_px, y_px, r_px, x_unc, y_unc, r_eff_ss_unc, phi_unc, pd.to_datetime(file_name.replace(".jpg", "").replace("sunspots_1024_", ""))])
            
        df = pd.DataFrame(data=phis, columns=["x_py", "y_px", "r_py", "x", "y", "r_eff_ss", "phi", "day"])
        out_filename = f"phi_{file_name.replace('.jpg', '.csv')}"
        df.to_csv(out_filename, sep=";")
        print(f"Saved data to {out_filename}")
    else:
        print("No circles found")

In [ ]:
img = read_file(DATA_PATH, TEST_FILE, show=True)

In [ ]:
circles = get_circles(img, TEST_FILE, show=True)

In [ ]:
calc_phi(circles, TEST_FILE)

In [ ]:
files = [f for f in os.listdir(DATA_PATH) if f.endswith(".jpg")]

for f in files:
    print(f"===\nFilename: {f}")
    img = read_file(DATA_PATH, f)
    circles = get_circles(img, f)
    print(calc_phi(circles, f))